In [ ]:
import os
from pprint import pprint
import torch
from tiatoolbox.wsicore.wsireader import VirtualWSIReader, WSIReader
from tiatoolbox.tools.patchextraction import get_patch_extractor
from tissue_masker_lite import get_mask
import matplotlib.pyplot as plt
import numpy as np
from tiatoolbox.models.engine.semantic_segmentor import SemanticSegmentor
import segmentation_models_pytorch as smp
from tqdm.auto import tqdm
import cv2
import pickle
import skimage
import cv2
import skimage.measure

In [ ]:
wsi_name = "105S.tif"
wsi_without_ext = os.path.splitext(wsi_name)[0]
masks_dir = "/home/u1910100/GitHub/TIAger-Torch/output/seg_out"
tumor_stroma_mask_path = os.path.join(masks_dir, f"{wsi_without_ext}_tumor_stroma.npy")

tumor_stroma_mask = np.load(tumor_stroma_mask_path)

plt.imshow(tumor_stroma_mask)
plt.show()

In [ ]:
wsi_dir = "/home/u1910100/Documents/Tiger_Data/wsitils/images/"
wsi_path = os.path.join(wsi_dir, wsi_name)
wsi = WSIReader.open(wsi_path)

patch_extractor = get_patch_extractor(
    input_img=wsi,
    method_name="slidingwindow",
    patch_size=(1024, 1024),
    resolution=20,
    units="power",
    input_mask=tumor_stroma_mask,
)

patch_1 = patch_extractor[0]

In [ ]:
def count_tils_in_tile(tile_image):
    tile_reader = VirtualWSIReader(tile_image, power=20)

    patch_extractor = get_patch_extractor(
        input_img=tile_reader,
        method_name="slidingwindow",
        patch_size=(128, 128),
        stride=(28,28),
        resolution=20,
        units="power",
    )

    print(len(patch_extractor))
    for i in range(0,50):
        plt.imshow(patch_extractor[i])
        plt.show()

In [ ]:
count_tils_in_tile(patch_1)